<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


 ## Lab 2: Data wrangling 


Estimated time needed: **60** minutes


In this lab, we will perform some Exploratory Data Analysis (EDA) to find some patterns in the data and determine what would be the label for training supervised models. 

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident; for example, <code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully landed on  a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed on a drone ship. 

In this lab we will mainly convert those outcomes into Training Labels with `1` means the booster successfully landed `0` means it was unsuccessful.


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objectives
Perform exploratory  Data Analysis and determine Training Labels 

- Exploratory Data Analysis
- Determine Training Labels 


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [3]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis 


Load Space X dataset, from last section.


In [4]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identify and calculate the percentage of the missing values in each attribute


In [5]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identify which columns are numerical and categorical:


In [6]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### TASK 1: Calculate the number of launches on each site

The data contains several Space X  launch facilities: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>Cape Canaveral Space</a> Launch Complex 40  <b>VAFB SLC 4E </b> , Vandenberg Air Force Base Space Launch Complex 4E <b>(SLC-4E)</b>, Kennedy Space Center Launch Complex 39A <b>KSC LC 39A </b>.The location of each Launch Is placed in the column <code>LaunchSite</code>


Next, let's see the number of launches for each site.

Use the method  <code>value_counts()</code> on the column <code>LaunchSite</code> to determine the number of launches  on each site: 


In [30]:
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Step 2: Get rocket names using a second API call
rockets_url = "https://api.spacexdata.com/v4/rockets"
rockets_response = requests.get(rockets_url)
rockets_data = rockets_response.json()

# Convert the rockets data to a DataFrame
rockets_df = pd.json_normalize(rockets_data)

# Create a dictionary to map rocket IDs to their names
rocket_id_to_name = rockets_df.set_index('id')['name'].to_dict()

# Add a new column for rocket names to the main DataFrame
df['rocket_name'] = df['rocket'].map(rocket_id_to_name)

# Step 3: Filter out Falcon 1 launches to count the Falcon 9 launches
falcon_9_launches = df[df['rocket_name'] == 'Falcon 9']
falcon_9_count = falcon_9_launches.shape[0]

# Step 4: Count missing values in the `launchpad` column
missing_landing_pad_count = df['launchpad'].isnull().sum()

# Step 5: Extract the year from the first row of the `static_fire_date_utc` column
first_row_year = pd.to_datetime(df.loc[0, 'static_fire_date_utc']).year

# Step 6: Calculate the number of launches on each site
launch_site_counts = df['launchpad'].value_counts()

# Output the results
print(f"Number of Falcon 9 launches: {falcon_9_count}")
print(f"Number of missing values in 'launchpad' column: {missing_landing_pad_count}")
print(f"Year in the first row of 'static_fire_date_utc': {first_row_year}")
print("Number of launches on each site:")
print(launch_site_counts)

Number of Falcon 9 launches: 195
Number of missing values in 'launchpad' column: 0
Year in the first row of 'static_fire_date_utc': 2006
Number of launches on each site:
5e9e4501f509094ba4566f84    112
5e9e4502f509094188566f88     58
5e9e4502f509092b78566f87     30
5e9e4502f5090995de566f86      5
Name: launchpad, dtype: int64


Each launch aims to an dedicated orbit, and here are some common orbit types:




* <b>LEO</b>: Low Earth orbit (LEO)is an Earth-centred orbit with an altitude of 2,000 km (1,200 mi) or less (approximately one-third of the radius of Earth),[1] or with at least 11.25 periods per day (an orbital period of 128 minutes or less) and an eccentricity less than 0.25.[2] Most of the manmade objects in outer space are in LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Very Low Earth Orbits (VLEO) can be defined as the orbits with a mean altitude below 450 km. Operating in these orbits can provide a number of benefits to Earth observation spacecraft as the spacecraft operates closer to the observation<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.


* <b>GTO</b> A geosynchronous orbit is a high Earth orbit that allows satellites to match Earth's rotation. Located at 22,236 miles (35,786 kilometers) above Earth's equator, this position is a valuable spot for monitoring weather, communications and surveillance. Because the satellite orbits at the same speed that the Earth is turning, the satellite seems to stay in place over a single longitude, though it may drift north to south,” NASA wrote on its Earth Observatory website <a  href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.


* <b>SSO (or SO)</b>: It is a Sun-synchronous orbit  also called a heliosynchronous orbit is a nearly polar orbit around a planet, in which the satellite passes over any given point of the planet's surface at the same local mean solar time <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.
    
    
    
* <b>ES-L1 </b>:At the Lagrange points the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one such point between the sun and the earth <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .
    
    
* <b>HEO</b> A highly elliptical orbit, is an elliptic orbit with high eccentricity, usually referring to one around Earth <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.


* <b> ISS </b> A modular space station (habitable artificial satellite) in low Earth orbit. It is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada)<a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>


* <b> MEO </b> Geocentric orbits ranging in altitude from 2,000 km (1,200 mi) to just below geosynchronous orbit at 35,786 kilometers (22,236 mi). Also known as an intermediate circular orbit. These are "most commonly at 20,200 kilometers (12,600 mi), or 20,650 kilometers (12,830 mi), with an orbital period of 12 hours <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>


* <b> HEO </b> Geocentric orbits above the altitude of geosynchronous orbit (35,786 km or 22,236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>


* <b> GEO </b> It is a circular geosynchronous orbit 35,786 kilometres (22,236 miles) above Earth's equator and following the direction of Earth's rotation <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>


* <b> PO </b> It is one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited (usually a planet such as the Earth <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

some are shown in the following plot:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### TASK 2: Calculate the number and occurrence of each orbit


 Use the method  <code>.value_counts()</code> to determine the number and occurrence of each orbit in the  column <code>Orbit</code>


In [9]:
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Print the columns of the DataFrame to inspect the structure
print(df.columns)

# Check if 'orbit' or similar columns exist in the DataFrame
possible_orbit_columns = [col for col in df.columns if 'orbit' in col.lower()]
print(f"Possible orbit columns: {possible_orbit_columns}")

# If the correct column name is found, apply value_counts to that column
if possible_orbit_columns:
    orbit_column = possible_orbit_columns[0]
    orbit_counts = df[orbit_column].value_counts()
    print("Number and occurrence of each orbit:")
    print(orbit_counts)
else:
    print("No 'orbit' column found in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Possible orbit columns: []
No 'orbit' column found in the DataFrame.


### TASK 3: Calculate the number and occurence of mission outcome of the orbits


Use the method <code>.value_counts()</code> on the column <code>Outcome</code> to determine the number of <code>landing_outcomes</code>.Then assign it to a variable landing_outcomes.


In [10]:
# landing_outcomes = values on Outcome column
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Print the columns of the DataFrame to inspect the structure
print(df.columns)

# Check if 'outcome' or similar columns exist in the DataFrame
possible_outcome_columns = [col for col in df.columns if 'outcome' in col.lower()]
print(f"Possible outcome columns: {possible_outcome_columns}")

# If the correct column name is found, apply value_counts to that column
if possible_outcome_columns:
    outcome_column = possible_outcome_columns[0]
    landing_outcomes = df[outcome_column].value_counts()
    print("Number and occurrence of each landing outcome:")
    print(landing_outcomes)
else:
    print("No 'outcome' column found in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Possible outcome columns: []
No 'outcome' column found in the DataFrame.


<code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully  landed to a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed to a drone ship. <code>None ASDS</code> and <code>None None</code> these represent a failure to land.


In [14]:
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Step 2: Inspect the DataFrame columns to identify the correct column for landing outcomes
print(df.columns)

# Assuming the correct column is 'cores.landing_success' for landing outcomes
# If this is not correct, inspect the printed columns and adjust accordingly
if 'cores' in df.columns:
    df = df.explode('cores')
    df['landing_success'] = df['cores'].apply(lambda x: x['landing_success'] if isinstance(x, dict) else None)

    # Step 3: Calculate the value counts for the landing outcomes
    landing_outcomes = df['landing_success'].value_counts(dropna=False)
    print("Number and occurrence of each landing outcome:")
    print(landing_outcomes)

    # Step 4: Iterate through the landing outcomes and print the index and outcome
    for i, outcome in enumerate(landing_outcomes.keys()):
        print(i, outcome)
else:
    print("The 'cores' column was not found in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number and occurrence of each landing outcome:
True     149
NaN       53
False     13
Name: landing_success, dtype: int64
0 True
1 nan
2 False


In [12]:
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Step 2: Inspect the DataFrame columns to identify the correct column for landing outcomes
print(df.columns)

# Assume the column 'landing_outcome' contains the landing outcomes
# If 'landing_outcome' is not found, identify the correct column by inspecting the printed columns
possible_outcome_columns = [col for col in df.columns if 'landing' in col.lower()]
print(f"Possible outcome columns: {possible_outcome_columns}")

if possible_outcome_columns:
    outcome_column = possible_outcome_columns[0]

    # Step 3: Calculate the value counts for the landing outcomes
    landing_outcomes = df[outcome_column].value_counts()
    print("Number and occurrence of each landing outcome:")
    print(landing_outcomes)

    # Step 4: Iterate through the landing outcomes and print the index and outcome
    for i, outcome in enumerate(landing_outcomes.keys()):
        print(i, outcome)
else:
    print("No landing outcome column found in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Possible outcome columns: []
No landing outcome column found in the DataFrame.


We create a set of outcomes where the second stage did not land successfully:


In [16]:
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Step 2: Inspect the DataFrame columns to identify the correct column for landing outcomes
print(df.columns)

# Assuming the correct column is 'cores.landing_success' for landing outcomes
# If this is not correct, inspect the printed columns and adjust accordingly
if 'cores' in df.columns:
    df = df.explode('cores')
    df['landing_success'] = df['cores'].apply(lambda x: x['landing_success'] if isinstance(x, dict) else None)

    # Step 3: Calculate the value counts for the landing outcomes
    landing_outcomes = df['landing_success'].value_counts(dropna=False)
    print("Number and occurrence of each landing outcome:")
    print(landing_outcomes)

    # Step 4: Print the length and content of landing_outcomes.keys()
    landing_outcome_keys = landing_outcomes.keys()
    print("Landing outcome keys:")
    for i, key in enumerate(landing_outcome_keys):
        print(f"{i}: {key}")

    # Adjust indices based on the actual length of landing_outcome_keys
    valid_indices = [i for i in [1, 3, 5, 6, 7] if i < len(landing_outcome_keys)]
    
    # Step 5: Select bad outcomes based on valid indices
    bad_outcomes = set(landing_outcome_keys[valid_indices])
    print("Bad outcomes:")
    print(bad_outcomes)
else:
    print("The 'cores' column was not found in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number and occurrence of each landing outcome:
True     149
NaN       53
False     13
Name: landing_success, dtype: int64
Landing outcome keys:
0: True
1: nan
2: False
Bad outcomes:
{nan}


### TASK 4: Create a landing outcome label from Outcome column


Using the <code>Outcome</code>,  create a list where the element is zero if the corresponding  row  in  <code>Outcome</code> is in the set <code>bad_outcome</code>; otherwise, it's one. Then assign it to the variable <code>landing_class</code>:


In [17]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
import requests
import pandas as pd

# Step 1: Get the data from the SpaceX API
spacex_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_url)
data = response.json()

# Convert the response to a DataFrame
df = pd.json_normalize(data)

# Step 2: Inspect the DataFrame columns to identify the correct column for landing outcomes
print(df.columns)

# Assuming the correct column is 'cores.landing_success' for landing outcomes
if 'cores' in df.columns:
    df = df.explode('cores')
    df['landing_success'] = df['cores'].apply(lambda x: x['landing_success'] if isinstance(x, dict) else None)

    # Step 3: Calculate the value counts for the landing outcomes
    landing_outcomes = df['landing_success'].value_counts(dropna=False)
    print("Number and occurrence of each landing outcome:")
    print(landing_outcomes)

    # Step 4: Print the length and content of landing_outcomes.keys()
    landing_outcome_keys = landing_outcomes.keys()
    print("Landing outcome keys:")
    for i, key in enumerate(landing_outcome_keys):
        print(f"{i}: {key}")

    # Adjust indices based on the actual length of landing_outcome_keys
    valid_indices = [i for i in [1, 3, 5, 6, 7] if i < len(landing_outcome_keys)]
    
    # Step 5: Select bad outcomes based on valid indices
    bad_outcomes = set(landing_outcome_keys[valid_indices])
    print("Bad outcomes:")
    print(bad_outcomes)
    
    # Step 6: Create a landing outcome label from Outcome column
    landing_class = df['landing_success'].apply(lambda x: 0 if x in bad_outcomes else 1)
    
    # Assign landing_class to the DataFrame
    df['landing_class'] = landing_class

    # Display the DataFrame with the new landing_class column
    print(df[['landing_success', 'landing_class']].head())
else:
    print("The 'cores' column was not found in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number and occurrence of each landing outcome:
True     149
NaN       53
False     13
Name: landing_success, dtype: int64
Landing outcome keys:
0: True
1: nan
2: False
Bad outcomes:
{nan}
  landing_success  la

This variable will represent the classification variable that represents the outcome of each launch. If the value is zero, the  first stage did not land successfully; one means  the first stage landed Successfully 


In [18]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1


In [19]:
df.head(5)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings,landing_success,landing_class,Class
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN,None,1,1
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN,None,1,1
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN,None,1,1
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN,None,1,1
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN,None,1,1


We can use the following line of code to determine  the success rate:


In [20]:
df["Class"].mean()

1.0

We can now export it to a CSV for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |


Copyright © 2021 IBM Corporation. All rights reserved.
